In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [35]:
import itertools
import re

In [31]:
POPDATA = "data/canada_pop-dwelling_by_postalcode_2011.csv"
BASICBIZ = "yelp_total.csv"

In [23]:
pops = pd.read_csv(POPDATA, index_col="Geographic name")

In [24]:
pops.head()

,"Population, 2011","Total private dwellings, 2011","Private dwellings occupied by usual residents, 2011"
Geographic name,,,
A0A,46297,23950,18701
A0B,20985,12585,8854
A0C,12834,8272,5482
A0E,23384,12733,9659
A0G,36264,21153,14967


In [90]:
pops.columns

Index([u'Population, 2011', u'Total private dwellings, 2011',
       u'Private dwellings occupied by usual residents, 2011'],
      dtype='object')

In [25]:
nums = ["1", "2", "3", "4", "6", "7", "9"]
letts = "ABCEGHJKLMNPRSTVWXYZ"
MTL_POSTCODES = set(['H%s%s' % xx for xx in itertools.product(nums, letts)])
MTL_POSTCODES.update(["H5A", "H5B", "H0M", "H8N", "H8P", "H8R", "H8S", "H8T", "H8Y", "H8Z"])
MTL_POSTCODES.difference_update(["H7Z", "H9Y", "H9Z", "H9V", "H9T", "H9N", "H9M", "H9L"])

In [26]:
MTL_POSTCODES = sorted(list(MTL_POSTCODES))

Filter to include only Montreal

In [29]:
pops = pops.loc[list(MTL_POSTCODES)]

In [30]:
pops.head()

,"Population, 2011","Total private dwellings, 2011","Private dwellings occupied by usual residents, 2011"
Geographic name,,,
H0M,NaN,NaN,NaN
H1A,32432,14471,13885
H1B,19854,9171,8772
H1C,13659,4384,4273
H1E,43294,17238,16630


Read in businesses

In [32]:
business = pd.read_csv(BASICBIZ, index_col='business_id')

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (79,91,93,94,95,96,109) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [51]:
postcode_re = re.compile("QC (?P<fsa>H[0-9][A-Z]) (?P<ldu>[0-9][A-Z][0-9])")

In [52]:
match = postcode_re.search(business.full_address.ix[0])
print match.groupdict()

{'fsa': 'H1W', 'ldu': '1R7'}


In [70]:
matches = business.full_address.str.findall(postcode_re)

In [77]:
fsas = matches.str.get(0).str.get(0)

In [81]:
fsas.head()

business_id
lW9Up8oVTWUEut_R6nZG-g    H1W
RZma06CqXbOIMzTQL-eJFQ    NaN
GxfgoiDLfs7HC8C_wFsGPw    H1Y
yLj-pKIc-V530j1G6ySQ2Q    H1V
IY5GFZTZCywC0ylKuG9ebA    H1W
Name: full_address, dtype: object

In [88]:
wpop = business.merge(pops, how='left', left_on=fsas, right_index=True)

In [92]:
popsonly = wpop.loc[:, [u'Population, 2011', u'Total private dwellings, 2011',
       u'Private dwellings occupied by usual residents, 2011']]

In [96]:
popsonly.rename(columns=dict(zip([u'Population, 2011', u'Total private dwellings, 2011',
       u'Private dwellings occupied by usual residents, 2011'], ('pop', 'homes', 'homes_occupied_by_owners'))), inplace=True)

In [98]:
popsonly.to_csv("data/business_local_population.csv")

In [99]:
!head data/business_local_population.csv

business_id,pop,homes,homes_occupied_by_owners
lW9Up8oVTWUEut_R6nZG-g,28422.0,16429.0,15307.0
RZma06CqXbOIMzTQL-eJFQ,,,
GxfgoiDLfs7HC8C_wFsGPw,25836.0,14357.0,13749.0
yLj-pKIc-V530j1G6ySQ2Q,21093.0,12097.0,11357.0
IY5GFZTZCywC0ylKuG9ebA,28422.0,16429.0,15307.0
RySnetMQUJKbbtliTt4Nmw,21093.0,12097.0,11357.0
IYdjcA92wjvM5RtPZ3C7kw,28928.0,14958.0,14276.0
pvbBfLIVJ_5sJXUTAKmkng,24490.0,10638.0,10177.0
A9yOopsy2bQgStF3Y6lSyA,21093.0,12097.0,11357.0
